In [1]:
from birdcall.data import *
from birdcall.metrics import *
from birdcall.ops import *

import torch
import torchvision
from torch import nn
import numpy as np
import pandas as pd
from pathlib import Path
import soundfile as sf

In [2]:
BS = 32
MAX_LR = 2e-3

In [3]:
classes = pd.read_pickle('data/classes.pkl')

In [4]:
from pathlib import Path
import soundfile as sf

In [5]:
splits = pd.read_pickle('data/all_splits.pkl')
all_train_items = pd.read_pickle('data/all_train_items.pkl')

train_items = np.array(all_train_items)[splits[0][0]].tolist()
val_items = np.array(all_train_items)[splits[0][1]].tolist()

In [6]:
from collections import defaultdict

class2train_items = defaultdict(list)

for cls_name, path, duration in train_items:
    class2train_items[cls_name].append((path, duration))

In [7]:
train_ds = MelspecPoolDataset(class2train_items, classes, len_mult=50, normalize=False)
train_dl = torch.utils.data.DataLoader(train_ds, batch_size=BS, num_workers=NUM_WORKERS, pin_memory=True, shuffle=True)

In [8]:
val_items = [(classes.index(item[0]), item[1], item[2]) for item in val_items]
val_items_binned = bin_items_negative_class(val_items)

In [8]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.cnn = nn.Sequential(*list(torchvision.models.resnet34(True).children())[:-2])
        self.classifier = nn.Sequential(*[
            nn.Linear(512, 512), nn.ReLU(), nn.Dropout(p=0.5), nn.BatchNorm1d(512),
            nn.Linear(512, 512), nn.ReLU(), nn.Dropout(p=0.5), nn.BatchNorm1d(512),
            nn.Linear(512, len(classes))
        ])
    
    def forward(self, x):
        max_per_example = x.view(x.shape[0], -1).max(1)[0] # scaling to between 0 and 1
        x /= max_per_example[:, None, None, None, None]     # per example!
        bs, im_num = x.shape[:2]
        x = x.view(-1, x.shape[2], x.shape[3], x.shape[4])
        x = self.cnn(x)
        x = x.mean((2,3))
        x = self.classifier(x)
        x = x.view(bs, im_num, -1)
        x = lme_pool(x)
        return x

In [9]:
model = Model().cuda()

In [10]:
import torch.optim as optim
from sklearn.metrics import accuracy_score, f1_score
import time

In [11]:
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), 1e-3)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, 5)

In [ ]:
t0 = time.time()
for epoch in range(130):
    running_loss = 0.0
    for i, data in enumerate(train_dl, 0):
        model.train()
        inputs, labels = data[0].cuda(), data[1].cuda()
        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        if np.isnan(loss.item()): 
            raise Exception(f'!!! nan encountered in loss !!! epoch: {epoch}\n')
        loss.backward()
        optimizer.step()
        scheduler.step()

        running_loss += loss.item()


    if epoch % 5 == 4:
        model.eval();
        preds = []
        targs = []

        for num_specs in val_items_binned.keys():
            valid_ds = MelspecShortishValidatioDataset(val_items_binned[num_specs], classes)
            valid_dl = torch.utils.data.DataLoader(valid_ds, batch_size=2*BS, num_workers=NUM_WORKERS, pin_memory=True)

            with torch.no_grad():
                for data in valid_dl:
                    inputs, labels = data[0].cuda(), data[1].cuda()
                    outputs = model(inputs)
                    preds.append(outputs.cpu().detach())
                    targs.append(labels.cpu().detach())

        preds = torch.cat(preds)
        targs = torch.cat(targs)

        accuracy = accuracy_score(preds.sigmoid() > 0.5, targs)
        f1 = f1_score(preds.sigmoid() > 0.5, targs, average='micro')
        print(f'[{epoch + 1}, {(time.time() - t0)/60:.1f}] loss: {running_loss / (len(train_dl)-1):.3f}, acc: {accuracy:.3f}, f1: {f1:.3f}')
        running_loss = 0.0

        torch.save(model.state_dict(), f'models/{epoch+1}_lmepool_simple_log_{round(f1, 2)}.pth')

[5, 17.2] loss: 0.023, acc: 0.000, f1: 0.004
[10, 34.7] loss: 0.019, acc: 0.003, f1: 0.006
[15, 51.7] loss: 0.017, acc: 0.038, f1: 0.077
[20, 69.0] loss: 0.014, acc: 0.149, f1: 0.263
[25, 86.0] loss: 0.012, acc: 0.253, f1: 0.398
[30, 103.2] loss: 0.010, acc: 0.334, f1: 0.488
[35, 120.5] loss: 0.009, acc: 0.407, f1: 0.565
[40, 138.3] loss: 0.008, acc: 0.434, f1: 0.594
[45, 155.7] loss: 0.008, acc: 0.467, f1: 0.617
[50, 173.1] loss: 0.006, acc: 0.502, f1: 0.650
[55, 190.5] loss: 0.005, acc: 0.511, f1: 0.655
[60, 207.8] loss: 0.006, acc: 0.500, f1: 0.644
[65, 225.1] loss: 0.004, acc: 0.540, f1: 0.681
[70, 242.5] loss: 0.004, acc: 0.549, f1: 0.685
[75, 259.8] loss: 0.004, acc: 0.501, f1: 0.646
[80, 277.0] loss: 0.004, acc: 0.572, f1: 0.704
[85, 294.5] loss: 0.003, acc: 0.560, f1: 0.695
[90, 313.1] loss: 0.003, acc: 0.582, f1: 0.713
[95, 331.1] loss: 0.003, acc: 0.574, f1: 0.704
[100, 349.8] loss: 0.003, acc: 0.581, f1: 0.709
[105, 367.5] loss: 0.002, acc: 0.576, f1: 0.702
[110, 386.0] loss

In [12]:
model.load_state_dict(torch.load('models/130_lmepool_simple_log_0.71.pth'))

<All keys matched successfully>

In [26]:
model.eval();
preds = []
targs = []

for num_specs in val_items_binned.keys():
    valid_ds = MelspecShortishValidatioDataset(val_items_binned[num_specs], classes)
    valid_dl = torch.utils.data.DataLoader(valid_ds, batch_size=2*BS, num_workers=NUM_WORKERS, pin_memory=True)

    with torch.no_grad():
        for data in valid_dl:
            inputs, labels = data[0].cuda(), data[1].cuda()
            outputs = model(inputs)
            preds.append(outputs.cpu().detach())
            targs.append(labels.cpu().detach())

preds = torch.cat(preds)
targs = torch.cat(targs)

In [27]:
f1s = []
ts = []
for t in np.linspace(0.4, 1, 61):
    f1s.append(f1_score(preds.sigmoid() > t, targs, average='micro'))
    ts.append(t)

In [31]:
accuracy_score(preds.sigmoid() > ts[np.argmax(f1s)], targs), max(f1s)

(0.5817543859649122, 0.7097361237488625)

In [29]:
ts[np.argmax(f1s)]

0.4

In [30]:
preds_to_tp_fp_fn(preds, targs)

(tensor(2598), tensor(469), tensor(1677))

In [13]:
sc_ds = SoundscapeMelspecPoolDataset(pd.read_pickle('data/soundscape_items.pkl'), classes)
sc_dl = torch.utils.data.DataLoader(sc_ds, batch_size=2*BS, num_workers=NUM_WORKERS, pin_memory=True)

In [14]:
t0 = time.time()
for epoch in range(130, 260):
    running_loss = 0.0
    for i, data in enumerate(train_dl, 0):
        model.train()
        inputs, labels = data[0].cuda(), data[1].cuda()
        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        if np.isnan(loss.item()): 
            raise Exception(f'!!! nan encountered in loss !!! epoch: {epoch}\n')
        loss.backward()
        optimizer.step()
        scheduler.step()

        running_loss += loss.item()


    if epoch % 5 == 4:
        model.eval();
        preds = []
        targs = []

        for num_specs in val_items_binned.keys():
            valid_ds = MelspecShortishValidatioDataset(val_items_binned[num_specs], classes)
            valid_dl = torch.utils.data.DataLoader(valid_ds, batch_size=2*16, num_workers=NUM_WORKERS//2, pin_memory=True)

            with torch.no_grad():
                for data in valid_dl:
                    inputs, labels = data[0].cuda(), data[1].cuda()
                    outputs = model(inputs)
                    preds.append(outputs.cpu().detach())
                    targs.append(labels.cpu().detach())

        preds = torch.cat(preds)
        targs = torch.cat(targs)

        f1s = []
        ts = []
        for t in np.linspace(0.4, 1, 61):
            f1s.append(f1_score(preds.sigmoid() > t, targs, average='micro'))
            ts.append(t)
        
        sc_preds = []
        sc_targs = []
        with torch.no_grad():
            for data in sc_dl:
                inputs, labels = data[0].cuda(), data[1].cuda()
                outputs = model(inputs)
                sc_preds.append(outputs.cpu().detach())
                sc_targs.append(labels.cpu().detach())

        sc_preds = torch.cat(sc_preds)
        sc_targs = torch.cat(sc_targs)
        sc_f1 = f1_score(sc_preds.sigmoid() > 0.5, sc_targs, average='micro')
        
        sc_f1s = []
        sc_ts = []
        for t in np.linspace(0.4, 1, 61):
            sc_f1s.append(f1_score(sc_preds.sigmoid() > t, sc_targs, average='micro'))
            sc_ts.append(t)
        
        f1 = f1_score(preds.sigmoid() > 0.5, targs, average='micro')
        print(f'[{epoch + 1}, {(time.time() - t0)/60:.1f}] loss: {running_loss / (len(train_dl)-1):.3f}, f1: {max(f1s):.3f}, sc_f1: {max(sc_f1s):.3f}')
        running_loss = 0.0

        torch.save(model.state_dict(), f'models/{epoch+1}_lmepool_simple_minmax_{round(f1, 2)}.pth')

[135, 18.3] loss: 0.002, acc: 0.599, f1: 0.725, sc_f1: 0.019


KeyboardInterrupt: 

[135, 16.9] loss: 0.001, f1: 0.725, sc_f1: 0.035
[140, 35.0] loss: 0.001, f1: 0.722, sc_f1: 0.026
[145, 53.7] loss: 0.001, f1: 0.724, sc_f1: 0.018
[150, 73.5] loss: 0.001, f1: 0.726, sc_f1: 0.011
[160, 110.4] loss: 0.001, f1: 0.729, sc_f1: 0.020
[165, 129.4] loss: 0.001, f1: 0.729, sc_f1: 0.011
[170, 147.4] loss: 0.001, f1: 0.726, sc_f1: 0.021
[175, 167.7] loss: 0.001, f1: 0.731, sc_f1: 0.022
[180, 185.0] loss: 0.001, f1: 0.726, sc_f1: 0.032
[185, 203.2] loss: 0.001, f1: 0.722, sc_f1: 0.023
[190, 221.2] loss: 0.001, f1: 0.730, sc_f1: 0.011
[195, 239.5] loss: 0.001, f1: 0.723, sc_f1: 0.011
[200, 259.9] loss: 0.001, f1: 0.719, sc_f1: 0.012
[205, 278.3] loss: 0.001, f1: 0.725, sc_f1: 0.023
[210, 298.2] loss: 0.001, f1: 0.736, sc_f1: 0.021
[215, 317.0] loss: 0.001, f1: 0.728, sc_f1: 0.026
[220, 337.0] loss: 0.001, f1: 0.732, sc_f1: 0.012
[225, 356.3] loss: 0.001, f1: 0.727, sc_f1: 0.012
[230, 376.2] loss: 0.001, f1: 0.734, sc_f1: 0.017
[235, 395.0] loss: 0.001, f1: 0.728, sc_f1: 0.012
[240, 413.7] loss: 0.001, f1: 0.728, sc_f1: 0.012
[245, 432.7] loss: 0.001, f1: 0.732, sc_f1: 0.020
[250, 451.6] loss: 0.001, f1: 0.729, sc_f1: 0.022
[255, 470.0] loss: 0.001, f1: 0.725, sc_f1: 0.011
[260, 490.0] loss: 0.001, f1: 0.722, sc_f1: 0.010